In [ ]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from gurobi import *

np.set_printoptions(precision=3)
pd.options.display.max_columns = None
from IPython.display import display, HTML
from IPython.display import display_html

sns.set(font_scale=1.5)
sns.set_palette(sns.color_palette("colorblind", 10))
sns.set_style("whitegrid")
sns.set_context("talk")

from dynamic_matching_solvers import *

## First create some visualization functions for better understanding how the solvers are performing

In [ ]:
def display_3D(tableaus_int, tableaus_float, names, c, k, T, alphas):
    '''Given an array of numpy arrays, transforms to dataframes and displays'''
    
    metadf = pd.DataFrame({
        'copies' :[c[i] for i in range(allocs.shape[0])],
        'use times' :[k[i] for i in range(allocs.shape[0])]
    })
    
    html_str = ('<th>' 
                + ''.join([f'<td style="text-align:center">{name}</td>' for name in names])
                + '</th>')

    int_ixs = len(tableaus_int)-1
    tableaus_int.extend(tableaus_float)
    
    for i in range(T):
        row_dfs = [pd.DataFrame(tableau[:,:,i]) for tableau in tableaus_int]
    
        html_str += ('<tr>' + "<td style='vertical-align:center'> t = {}".format(i) +
                     f"<td style='vertical-align:top'>{metadf.to_html(index=False, float_format='%i')}</td>" +
                     ''.join(f"<td style='vertical-align:top'> {df.to_html(index=False, float_format=ix_to_format(ix,int_ixs))}</td>"
                             for ix,df in enumerate(row_dfs)) + 
                     '</tr>')
        
    
    html_str = f'<table>{html_str}</table>'
    
    
    columns=["t="+str(t) for t in range(T)]
    index=["i="+str(i) for i in range(I)]
    dfa = pd.DataFrame(alphas.reshape(I,T),columns=columns, index=index)

    columns = ["j="+str(j) for j in range(J)]
    dfb = pd.DataFrame(betas.reshape(1,J),columns=columns)

    html_str += f'<table><th><td style="text-align:center">alpha[i,t]</td><td style="text-align:center">beta[j]</td></th><tr><td></td><td>{dfa.to_html()}</td><td>{dfb.to_html()}</td></tr></table>' 
    
    display_html(html_str, raw=True) 

    
def ix_to_format(ix,intix):
    if ix > intix:
        return '%.3e'
    elif ix > intix:
        return '%.3f'
    else:
        return '%i'
        
def check_comp_terms(I,J,T,alphas, betas):
    
    comp_terms = np.zeros((I,J,T))
    for t in range(T):
        for j in range(J):
            for i in range(I):
                    comp_terms[i,j,t] = pairing_weights[i,j,t] - sum_alpha_it(alphas,i,t,k) - betas[j]                  
    return comp_terms

def check_comp_slackness(I,J,T,alphas,valid_matches, allocs):
    
    comp_slackness = np.zeros(valid_matches.shape)
    
    for t in range(T):
        for j in range(J):
            for i in range(I):
                    comp_slackness[i,j,t] = (pairing_weights[i,j,t] - sum_alpha_it(alphas,i,t,k) - betas[j]) * allocs[i,j,t]
                    
    return comp_slackness

def print_tableau(matches):
    for i in range(matches.shape[2]):
        print("t=",i)
        print(matches[:,:,i])

# Experiments calling the solvers defined above

### Comparing greedy vs. online dual vs. optimal in a slightly larger simulation

In [ ]:
def create_simulation_scenario(I,J,T, scenario=None, arrivals=''):
    "Defines tableau for 2 x 2 model according to arrivals and agent scenario distribution"
    if scenario:
        assert I == scenario['k'].size and I == scenario['c'].size and I == len(scenario['resource_configs'])+1, 'Scenario provided is not valid'
        assert arrivals != '', 'Arrival model not specified'
    
    valid_matches = np.zeros((I,J,T), np.int)
    weight_noise = np.random.random(valid_matches.shape) 

    #Only consider valid indicies
    for t in range(T):
        valid_matches[:,max(0,t-d):min(t+1,J),t] = 1

    if scenario == None:
        pairing_weights = np.random.random(valid_matches.shape) 

    #Construct matching matrix based on scenario configuration
    else:
        pairing_weights = np.zeros(valid_matches.shape)
        
        for resource_ix, config in enumerate(scenario['resource_configs']):
            
            if arrivals == "Alternating":
                agentAs = np.arange(J)[::2]
                agentBs = np.arange(J)[1::2]

            elif arrivals == "Stochastic":
                pA = scenario['resource_configs'][resource_ix]['probability-A']
                agentAs = np.arange(J)[np.random.choice([1, 0], size=J, p=[pA, 1-pA]) == 1]
                agentBs = np.setdiff1d(np.arange(J), agentAs)

            elif arrivals == "Adversarial":
                agentAs = np.arange(J)[0:int(J/2)]
                agentBs = np.setdiff1d(np.arange(J), agentAs)
                
            pairing_weights[resource_ix+1,agentAs,:] = scenario['resource_configs'][resource_ix]['agent-A']
            pairing_weights[resource_ix+1,agentBs,:] = scenario['resource_configs'][resource_ix]['agent-B']

    #Random noise to break ties
    for t in range(1,pairing_weights.shape[2]):
        pairing_weights[:,:,t] = pairing_weights[:,:,0] + .0001 * weight_noise[:,:,t]  
    
    #Modify pairing weights based on problem formulation 
    pairing_weights[0,:,:] =  0
    pairing_weights[valid_matches == 0] = -1
    pairing_weights[1:,:,:] = np.round(pairing_weights[1:,:,:],decimals=8)

    return valid_matches, pairing_weights

# Comparing performance of online-dual, greedy, offline 

In [ ]:
def compare_smr(I,J,T,arrivals='',scenario=None,plot=True):
    '''Run many simulations with different resource copy and use time properties. Plot performance as a function of resource scarcity'''
    primal_objs = []
    online_objs = []
    greedy_objs = []
    smrs = []

    for cmax in range(2,10):
        for kmin in range(4,12):
            for kmax in range(14,22):
                
                for p in range(12):

                    valid_matches, pairing_weights = create_simulation_scenario(I, J, T, scenario, arrivals)

                    k = np.insert(np.random.randint(kmin,kmax,size=I-1), 0, 1)
                    c = np.insert(np.random.randint(1,cmax,size=I-1), 0, J)

                    objp, allocs = primal_solutions(valid_matches, pairing_weights, I, J, T, k, c)
                    objd, alphas, betas = dual_solutions(valid_matches,pairing_weights, I, J, T, k, c)
                    objo, online_allocations = online_matching(I,J,T,k,c,alphas,betas,allocs,valid_matches,pairing_weights)
                    objg, greedy_allocs = greedy_matching(I,J,T,k,c,d,valid_matches,pairing_weights)

                    smr = np.round((J - allocs.sum()) / J, 3)

                    primal_objs.append(objp)
                    online_objs.append(objo)
                    greedy_objs.append(objg)
                    smrs.append(smr)

    if plot: 
        make_smr_plot(smrs, primal_objs, online_objs, greedy_objs)  
    
    return smrs, primal_objs, online_objs, greedy_objs

def make_smr_plot(smrs, primal_objs, online_objs, greedy_objs):
    '''Creates resource scarcity line plot'''
    smrt = 100 - (np.array(smrs) * 100)
    rundf = pd.DataFrame({'smrs': smrt, 'Offline Primal (Optimal)': primal_objs, 'Online Greedy': greedy_objs, 'Online Dual' : online_objs})
    plt.figure(figsize=(10,7))
    ax = sns.lineplot(x="smrs", y="value", data=pd.melt(rundf, ['smrs']), hue='variable')
    ax.set(xlabel='Percent of agents matched in optimal', ylabel='Match utility', title=f'Performance as a function of resource scarcity')

    legend = ax.legend()
    legend.texts[0].set_text("Algorithm")
    plt.show()
    
    
def compare_single_config(I, J, T, k, c, arrivals, scenario=None, plot=True, n_samples=40):
    '''For a single k,c configuration, run multiple simulations to compare performance distributions, optionally plotting the result'''
    primal_objs = []
    online_objs = []
    greedy_objs = []
    smrs = []

    for i in range(n_samples):

        valid_matches, pairing_weights = create_simulation_scenario(I, J, T, scenario, arrivals)

        objp, allocs = primal_solutions(valid_matches, pairing_weights, I, J, T, k, c)
        objd, alphas, betas = dual_solutions(valid_matches,pairing_weights, I, J, T, k, c)
        objo, online_allocations = online_matching(I,J,T,k,c,alphas,betas,allocs,valid_matches,pairing_weights)
        objg, greedy_allocs = greedy_matching(I,J,T,k,c,d,valid_matches,pairing_weights)

        smr = np.round((J - allocs.sum()) / J, 3)

        primal_objs.append(objp)
        online_objs.append(objo)
        greedy_objs.append(objg)
        smrs.append(smr)
    
    if plot:
        rundf= pd.DataFrame({'Primal': primal_objs, 'Online' : online_objs, 'Greedy': greedy_objs})
        plt.figure(figsize=(10,7))
        ax = sns.catplot(x="variable", y="value",  data=pd.melt(rundf))

        ax.set(xlabel='Matching algorithm', ylabel='Mean match utility', title=f'Average utility across 40 runs ({np.array(smrs).mean()} avg self match ratio, {J} agents, {I} resources, {d} round wait time)')
        plt.show()

    return smrs, primal_objs, online_objs, greedy_objs

def make_noise_plot(k,c,arrivals,scenariokey,scenario=None,):

    primal_objs = []
    online_objs = []
    greedy_objs = []
    smrs = []
    objos = []
    noises = []
    noise_terms = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 5e-1]

    for i in range(10):

        valid_matches, pairing_weights = create_simulation_scenario(I,J,T, scenario, arrivals)

        objp, allocs = primal_solutions(valid_matches, pairing_weights, I, J, T, k, c)
        objd, alphas, betas = dual_solutions(valid_matches,pairing_weights, I, J, T, k, c)
        objo, online_allocations = online_matching(I,J,T,k,c,alphas,betas,allocs,valid_matches,pairing_weights)
        objg, greedy_allocs = greedy_matching(I,J,T,k,c,d,valid_matches,pairing_weights)

        smr = np.round((J - allocs.sum()) / J, 3)

        primal_objs.append(objp)
        online_objs.append(objo)
        greedy_objs.append(objg)
        smrs.append(smr)

        for noise in noise_terms:
            for i in range(40):

                alphas_copy = alphas + noise * np.random.random(alphas.shape)
                objo, online_allocations = online_matching(I,J,T,k,c,alphas_copy,betas,allocs,valid_matches,pairing_weights)
                objos.append(objo)
                noises.append(noise)
                
    rundf= pd.DataFrame({'Primal': primal_objs, 'Greedy': greedy_objs, 'Online Optimal Dual' : online_objs})
    noisedf = pd.DataFrame({'noise_multiplier': noises, 'utility': objos})

    plt.figure(figsize=(10,7))
    fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True,gridspec_kw={'width_ratios': [1, 2]})
    fig.suptitle(f"Scenario {scenariokey} & {arrivals} Arrival Model".format(scenario,arrivals), fontsize=24)
    fig.set_figwidth(12)


    sns.catplot(x="variable", y="value",  data=pd.melt(rundf), kind='strip',ax=ax1,s=10)
    ax1.set(xlabel='', ylabel='Match utility', title='Baseline')
    plt.close() 

    sns.catplot(x="noise_multiplier", y="utility",  data=noisedf,  kind="strip",s=10,ax=ax2)    
    ax2.set(xlabel='Noise (%)', ylabel='', title='Online Dual with Increasing Noise')

    fig.autofmt_xdate(rotation=45)
    plt.tight_layout() 

    fig.subplots_adjust(top=0.75)
    plt.close() 
    

In [ ]:
# To create a simple scenario, run 

J=6
I=3
d=2
T = J+d 
valid_matches, pairing_weights = create_simulation_scenario(I,J,T)

k = np.array([1,3,3])
c = np.array([J,1, 1])

#To run the solvers, run: 
objp, allocs = primal_solutions(valid_matches, pairing_weights, I, J, T, k, c)
objd, alphas, betas = dual_solutions(valid_matches,pairing_weights, I, J, T, k, c)
objo, online_allocations = online_matching(I,J,T,k,c,alphas,betas,allocs,valid_matches,pairing_weights)
objg, greedy_allocs = greedy_matching(I,J,T,k,c,d,valid_matches,pairing_weights)

#You can then compare the allocations different algorithms make by, 
# display_3D([allocs, online_], tableaus_float, names, c, k, T, alphas)





## Exp 1: General comparison of online-dual, greedy, and primal

In [ ]:
J=4
I=3
d=2
T = J+d 

#Comparison across self-match-ratios
smrs, po2, oo2, go2 = compare_smr(I,J,T)    


In [ ]:
rundf= pd.DataFrame({'Offline Primal': po2,'Online Greedy': go2, 'Online Dual' : oo2})
plt.figure(figsize=(15,8))
ax = sns.catplot(x="variable", y="value",  data=pd.melt(rundf), kind='box')
ax.set(xlabel='Matching algorithm', ylabel='Match utility', title=f'Overall Matching Algorithm Performance')
ax.set_xticklabels(rotation=30)
plt.show()


## Exp 2: Deterioration of online-assignment with increasing noise

Define specific scenarios where a greedy approach will perform sub-optimally

In [ ]:
J=50
I=3
d=4
T = J+d 

scenarios = {}
scenarios['1'] = {}
scenarios['1']['k'] = np.array([1,5,15])
scenarios['1']['c'] =  np.array([J,10, 5])
scenarios['1']['resource_configs'] = []
scenarios['1']['resource_configs'].append({'agent-A': .05,'agent-B':.05,'probability-A': 1})
scenarios['1']['resource_configs'].append({'agent-A': .1,'agent-B':.9,'probability-A': .5})


scenarios['2'] = {}
scenarios['2']['k'] =  np.array([1,5,5])
scenarios['2']['c'] =  np.array([J,3, 3]) 
scenarios['2']['resource_configs'] = []
scenarios['2']['resource_configs'].append({'agent-A': .99,'agent-B':.99,'probability-A': 1})
scenarios['2']['resource_configs'].append({'agent-A': .95,'agent-B':.05,'probability-A': .5})


scenarios['3'] = {}
scenarios['3']['k'] = np.array([1,T,T])
scenarios['3']['c'] = np.array([J,2,2])
scenarios['3']['resource_configs'] = []
scenarios['3']['resource_configs'].append({'agent-A': .05,'agent-B':.05,'probability-A': 1})
scenarios['3']['resource_configs'].append({'agent-A': .1,'agent-B':.9,'probability-A': .5})


### Run a single scenario many times to get distribution comparison

In [ ]:
#Single config comparison
x,y,z = compare_single_config(I,J,T, k, c, arrivals='Stochastic', scenario=scenarios['1'], plot=True, n_samples=50)

### Make plot of performance deteriorating with noise

In [ ]:
for key, scenario in scenarios.items():
    for arrival in ['Alternating', 'Stochastic', 'Adversarial']:
        make_noise_plot(scenario['k'], scenario['c'], arrival, key, scenario)
        

## Visualizing the allocation scenario 

### Including the number of resource copies pending, alpha duals, match weights, etc

In [ ]:
def get_resource_uses_from_alloc(I,J,T,c,k,allocs):

    resources_used = np.zeros((I,T),dtype=np.float)
    resources_available = np.repeat(np.expand_dims(c,axis=1),T,axis=1)

    for t in range(T):
        for i in range(I):
            resources_used[i,t+1:t+k[i]-1] += allocs[i,:,t].sum()
            resources_available[i,t+1:t+k[i]] -= int(allocs[i,:,t].sum())
            
    return resources_used, resources_available

def plot_alloc_heatmap(weights, allocs, alphas, resources_available, resources_used):
    fig, (ax0, ax1, ax2,ax4) = plt.subplots(nrows=4, figsize=(18,10))

    ax0.set_title("Match value", fontsize=16, fontweight="bold")
    ax1.set_title("Alpha duals", fontsize=16, fontweight="bold")
    ax2.set_title("Number of resource copies available", fontsize=16, fontweight="bold")
    ax4.set_title("Primal Allocations", fontsize=16, fontweight="bold")
    
    
    sns.heatmap(weights[1:,:], cmap="BuPu", linewidths=.5,ax=ax0)
    sns.heatmap(alphas[1:,:], cmap="BuPu", linewidths=.5,ax=ax1)
    sns.heatmap(resources_available[1:,:], cmap="YlGnBu", linewidths=.5, ax=ax2)
    sns.heatmap(allocs[1:,:], cmap="BuPu", linewidths=.5,ax=ax4)
    
    fig.subplots_adjust(hspace=0.6)
    plt.show()

def simulate_average_resource_uses(I,J,T,d,arrival='', scenario=None, n_runs=1):
    
    allocs_runs = np.zeros((n_runs,I,J,T))
    weights_runs = np.zeros((n_runs,I,J,T))
    alphas_runs = np.zeros((n_runs,I,T))
    resources_available_runs = np.zeros((n_runs,I,T))
    resources_used_runs = np.zeros((n_runs,I,T))
    
    k = scenario['k']
    c = scenario['c']
    
    for run in range(n_runs):
        
        valid_matches, pairing_weights = create_simulation_scenario(I,J,T,scenario,arrival)

        objp, allocs = primal_solutions(valid_matches, pairing_weights, I, J, T, k, c)
        objd, alphas, betas = dual_solutions(valid_matches,pairing_weights, I, J, T, k, c)
        objo, online_allocations = online_matching(I,J,T,k,c,alphas,betas,allocs,valid_matches,pairing_weights)
        objg, greedy_allocs = greedy_matching(I,J,T,k,c,d,valid_matches,pairing_weights)

        resources_used, resources_available = get_resource_uses_from_alloc(I,J,T,c,k,allocs)
        
        allocs_runs[run,:,:,:] = allocs
        weights_runs[run,:,:,:] = pairing_weights
        alphas_runs[run,:,:]= alphas.reshape(I,T)
        resources_used_runs[run,:,:] = resources_used
        resources_available_runs[run,:,:] = resources_available

    weights_runs[weights_runs < 0] = 0
    
    return weights_runs.mean(axis=0).mean(axis=1), allocs_runs.mean(axis=0).mean(axis=1), alphas_runs.mean(axis=0), resources_used_runs.mean(axis=0), resources_available_runs.mean(axis=0)


## Visualizing the relationship between resource availability and alphas

### 30 agents of type A&B arrive in stochastic order. Below is matching for one randomly generated scenario. Y axis is resources of type each type, and X axis is t. Match value and primal allocatoins are averaged over J

In [ ]:
I=3
J=30
d=2
T=J+d

c = np.array([J,3,3])
k = np.array([1,5,5])

weights, allocs, alphas, resources_used, resources_available = simulate_average_resource_uses(I,J,T,d,arrival='Stochastic',scenario=scenarios['2'],n_runs=20)
plot_alloc_heatmap(weights, allocs, alphas, resources_available, resources_used)


### Same plot but with an alternating arrival order

In [ ]:

weights, allocs, alphas, resources_used, resources_available = simulate_average_resource_uses(I,J,T,d,arrival='Alternating',scenario=scenarios['2'],n_runs=1)
plot_alloc_heatmap(weights, allocs, alphas, resources_available, resources_used)

### Generating the scenario and running 20 times, averaging across runs. 

In [ ]:


weights, allocs, alphas, resources_used, resources_available = simulate_average_resource_uses(I,J,T,d,arrival='Stochastic',scenario=scenarios['2'],n_runs=20)
plot_alloc_heatmap(weights, allocs, alphas, resources_available, resources_used)


## Using the averaged alpha vector to make allocation decisions

- The below code will use run a simulation N_SIMULATION times, create an averaged alpha vector from all of these simulations, then use that average vector to conduct online matching. 

In [ ]:
display_dict = {'Baseline': 'Randomly generated', 'Stochastic':'Stochastic A&B', 'Alternating':'Alternating A&B','Adversarial':'Adversarial A&B', 'I6':'5 resources, alternating arrivals'}
def mean_alpha_vector(I,J,T, N_SIMULATION, N_TEST, arrivals='',scenario=None):
    alpha_runs = np.zeros((N_SIMULATION, I*T))
    alloc_runs = np.zeros((N_SIMULATION, I,J,T))
    
    k = scenario['k']
    c = scenario['c']
    

    #Simulate several runs to get an average alpha vector for the scenario
    for i in range(N_SIMULATION):
        valid_matches, pairing_weights = create_simulation_scenario(I, J, T, scenario, arrivals)

        objp, allocs = primal_solutions(valid_matches, pairing_weights, I, J, T, k, c)
        objd, alphas, betas = dual_solutions(valid_matches,pairing_weights, I, J, T, k, c)
        objo, online_allocations = online_matching(I,J,T,k,c,alphas,betas,allocs,valid_matches,pairing_weights)
        objg, greedy_allocs = greedy_matching(I,J,T,k,c,d,valid_matches,pairing_weights)

        alpha_runs[i,:] = alphas
        alloc_runs[i,:,:,:] = allocs

    mean_alphas = alpha_runs.mean(axis=0)
    mean_alloc = alloc_runs.mean(axis=0).mean(axis=1)
    
    return mean_alphas, mean_alloc


def simulate_alphas(I,J,T,N_SIMULATION,N_TEST,a=['Stochastic','Alternating','Adversarial'], scenario=None):
    
    objps = []
    objos = []
    objgs = []
    snrs = []
    arvls = []
    
    for arrivals in a:
        
        k = scenario['k']
        c = scenario['c']
        mean_alphas, mean_alloc = mean_alpha_vector(I,J,T, N_SIMULATION, N_TEST, arrivals, scenario)
       
        #Use the average vector to make allocation decisons several times
        for i in range(N_TEST):
            valid_matches, pairing_weights = create_simulation_scenario(I, J, T, scenario, arrivals)
            objp, allocs = primal_solutions(valid_matches, pairing_weights, I, J, T, k, c)
            objd, alphas, betas = dual_solutions(valid_matches,pairing_weights, I, J, T, k, c)
            objo, online_allocations = online_matching(I,J,T,k,c,mean_alphas,betas,allocs,valid_matches,pairing_weights)
            objg, greedy_allocs = greedy_matching(I,J,T,k,c,d,valid_matches,pairing_weights)

            objps.append(objp)
            objos.append(objo)
            objgs.append(objg)
            snrs.append(((J - allocs.sum()) / allocs.sum()))
            arvls.append(display_dict[arrivals])
            
            rundf = pd.DataFrame({'Offline Primal':objps, 'Online Avg Dual':objos, 'Online Greedy':objgs, 'snrs':snrs, 'arrivals':arvls})

    return rundf


In [ ]:
N_SIMULATION = 50
N_TEST = 30

d=2
J=30
I=3
T = J+d 

simdf = simulate_alphas(I,J,T,N_SIMULATION,N_TEST,a=['Stochastic','Alternating','Adversarial'], scenario=scenarios['3'])

plt.figure(figsize=(10,7))
ax = sns.catplot(x="variable", y="value", col='arrivals',  data=pd.melt(simdf,id_vars=['snrs','arrivals']), kind='violin')
ax.set_xticklabels(rotation=30)
